In [56]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import tqdm

from torchvision.datasets import FashionMNIST
from torchvision import transforms

fashion_mnist_train = FashionMNIST("./FashionMNIST",train=True,download=True,transform=transforms.ToTensor())
fashion_mnist_test = FashionMNIST("./FashionMNIST",train=False,download=True,transform=transforms.ToTensor())

batch_size = 128
train_loader = DataLoader(fashion_mnist_train,batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test,batch_size=batch_size, shuffle=False)

In [57]:
from torch import nn, optim

class FlattenLayer(nn.Module):
    def forward(self,x):
        sizes = x.size()
        return x.view(sizes[0], -1)

conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer(),
)

test_input = torch.ones(1,1,28,28)
conv_output_size = conv_net(test_input).size()[-1]

mlp = nn.Sequential(
    nn.Linear(conv_output_size,200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200,10)
)

net = nn.Sequential(
    conv_net,
    mlp
)
    

In [60]:
# 評価のヘルパー関数
def eval_net(net, data_loader, device="cpu"):
    # DropoutやBatchNormを無効化
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        # toメソッドで計算を実行するデバイスに転送する
        x = x.to(device)
        y = y.to(device)
        # 確率が最大のクラスを予測。(3章参照)
        # ここではfowardの計算だけなので自動部分に
        # 必要な余計な処理はoffにする
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    # ミニバッチごとの予測結果などを1つにまとめる
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 予測精度を計算
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

# 訓練のヘルパー関数
def train_net(net, train_loader, test_loader,
              optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),
              n_iter=10, device="cpu", writer=None):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # ネットワークを訓練モードにする
        net.train()
        n = 0
        n_acc = 0
        # 非常に時間がかかるのでtqdmを使用してプログレスバーを出す
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),
            total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        # 訓練データの予測精度
        train_acc.append(n_acc / n)
        # 検証データの予測精度
        val_acc.append(eval_net(net, test_loader, device))
        # このepochでの結果を表示
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)
        if writer is not None:
            writer.add_scalar('train_loss', train_losses[-1], epoch)
            writer.add_scalars('accuracy', {
            "train": train_acc[-1],
            "validation": val_acc[-1]
            }, epoch)
    

In [62]:


train_net(net, train_loader, test_loader, n_iter=20, device="cpu")


100%|██████████| 469/469 [00:19<00:00, 23.76it/s]


0 0.1523215719538494 0.9416666666666667 0.9207000136375427


100%|██████████| 469/469 [00:19<00:00, 24.10it/s]


1 0.14783135146443915 0.9445166666666667 0.9205999970436096


100%|██████████| 469/469 [00:19<00:00, 23.94it/s]


...

17 0.11632552545549524 0.9560166666666666 0.9218000173568726


100%|██████████| 469/469 [00:19<00:00, 24.19it/s]


18 0.11836416720468391 0.9563166666666667 0.9251000285148621


100%|██████████| 469/469 [00:20<00:00, 23.20it/s]


19 0.11131187988461083 0.9586833333333333 0.9247999787330627
